In [ ]:
import os
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

def create_directory_if_not_exists(directory_path):
    """指定されたパスにディレクトリが存在しない場合、新しいディレクトリを作成する"""
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"ディレクトリを作成しました: {directory_path}")
    else:
        print(f"ディレクトリは既に存在します: {directory_path}")
        
def load_video_info(filepath):
    """動画ファイルを読み込み、基本情報を返す"""
    cap = cv2.VideoCapture(filepath)
    if not cap.isOpened():
        raise ValueError("動画ファイルを開けませんでした: " + filepath)

    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    return cap, width, height, total_frame, fps

def detect_and_draw_contours(image_path, output_path):
    """画像の輪郭を検出し、描画して保存する"""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)
    cv2.imwrite(output_path, img)
    
    
def calculate_bubble_area(image_path):
    """画像内の気泡の面積を計算し、最大・最小面積を返す"""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    max_area = max(contours, key=cv2.contourArea)
    min_area = min(contours, key=cv2.contourArea)
    return cv2.contourArea(max_area), cv2.contourArea(min_area)


def save_expansion_contraction_images(max_area_frame, min_area_frame, input_path, output_path):
    """最大膨張時と最小収縮時の画像を保存する"""
    max_img = cv2.imread(f"{input_path}/rotated_frame_{max_area_frame:03d}.bmp")
    min_img = cv2.imread(f"{input_path}/rotated_frame_{min_area_frame:03d}.bmp")
    cv2.imwrite(f"{output_path}/max_expansion_{max_area_frame:03d}.bmp", max_img)
    cv2.imwrite(f"{output_path}/min_contraction_{min_area_frame:03d}.bmp", min_img)
